# Hierarchical Indices for RAG

## Setting Up the Environment

In [19]:
import numpy as np
import json
import fitz
import re
import pickle

In [20]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Document Processing Functions

In [21]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 페이지별로 텍스트와 메타데이터를 추출합니다.

    Args:
        pdf_path (str): PDF 파일 경로

    Returns:
        List[Dict]: 텍스트와 메타데이터를 포함한 페이지 목록
    """
    print(f"{pdf_path}에서 텍스트를 추출하는 중...")  # 처리 중인 PDF 경로 출력
    pdf = fitz.open(pdf_path)  # PyMuPDF를 사용하여 PDF 열기
    pages = []  # 추출된 페이지 데이터를 저장할 리스트 초기화

    # 각 페이지를 순회하며 텍스트 추출
    for page_num in range(len(pdf)):
        page = pdf[page_num]  # 현재 페이지 객체 가져오기
        text = page.get_text()  # 텍스트 추출

        # 텍스트가 50자 이하인 페이지는 건너뛰기
        if len(text.strip()) > 50:
            # 텍스트와 메타데이터를 포함하여 리스트에 추가
            pages.append({
                "text": text,
                "metadata": {
                    "source": pdf_path,       # 원본 파일 경로
                    "page": page_num + 1      # 페이지 번호 (1부터 시작)
                }
            })

    print(f"내용이 있는 페이지 {len(pages)}개를 추출했습니다.")  # 추출된 페이지 수 출력
    return pages  # 텍스트와 메타데이터를 포함한 페이지 리스트 반환

In [22]:
def chunk_text(text, metadata, chunk_size=1000, overlap=200):
    """
    텍스트를 일정 길이의 겹치는 청크로 나누고, 메타데이터를 함께 유지합니다.

    Args:
        text (str): 청크로 분할할 원본 텍스트
        metadata (Dict): 보존할 메타데이터 (예: 페이지 번호, 파일 경로 등)
        chunk_size (int): 각 청크의 문자 수
        overlap (int): 청크 간 중첩 문자 수

    Returns:
        List[Dict]: 각 청크와 메타데이터를 포함한 딕셔너리 리스트
    """
    chunks = []  # 생성된 청크들을 저장할 리스트 초기화

    # (chunk_size - overlap)만큼 이동하며 텍스트를 순회
    for i in range(0, len(text), chunk_size - overlap):
        chunk_text = text[i:i + chunk_size]  # 현재 위치에서 청크 텍스트 추출

        # 너무 짧은 청크는 건너뛰기 (50자 이하)
        if chunk_text and len(chunk_text.strip()) > 50:
            # 메타데이터 복사 후 청크별 정보 추가
            chunk_metadata = metadata.copy()
            chunk_metadata.update({
                "chunk_index": len(chunks),        # 청크 인덱스
                "start_char": i,                   # 원본 텍스트 기준 시작 위치
                "end_char": i + len(chunk_text),   # 원본 텍스트 기준 종료 위치
                "is_summary": False                # 요약 여부 플래그 (기본값 False)
            })

            # 청크와 메타데이터 함께 저장
            chunks.append({
                "text": chunk_text,
                "metadata": chunk_metadata
            })

    return chunks  # 청크 리스트 반환

## Simple Vector Store Implementation

In [23]:
class SimpleVectorStore:
    """
    NumPy를 사용한 간단한 벡터 저장소 구현 클래스입니다.
    """
    def __init__(self):
        self.vectors = []    # 임베딩 벡터들을 저장하는 리스트
        self.texts = []      # 해당 벡터에 대응되는 텍스트 콘텐츠
        self.metadata = []   # 텍스트에 대한 메타데이터
    
    def add_item(self, text, embedding, metadata=None):
        """
        벡터 저장소에 항목을 추가합니다.
        
        Args:
            text (str): 텍스트 콘텐츠
            embedding (List[float]): 텍스트에 대한 임베딩 벡터
            metadata (Dict, optional): 추가 메타데이터 (선택사항)
        """
        self.vectors.append(np.array(embedding))         # 임베딩 벡터를 NumPy 배열로 저장
        self.texts.append(text)                          # 텍스트 저장
        self.metadata.append(metadata or {})             # 메타데이터 저장 (없으면 빈 딕셔너리 사용)
    
    def similarity_search(self, query_embedding, k=5, filter_func=None):
        """
        쿼리 임베딩과 가장 유사한 항목들을 검색합니다.
        
        Args:
            query_embedding (List[float]): 쿼리 임베딩 벡터
            k (int): 반환할 결과 개수
            filter_func (callable, optional): 메타데이터 기반 필터링 함수 (선택사항)
        
        Returns:
            List[Dict]: 유사도가 높은 상위 k개의 항목 리스트
        """
        if not self.vectors:
            return []  # 저장된 벡터가 없으면 빈 리스트 반환
        
        # 쿼리 벡터를 NumPy 배열로 변환
        query_vector = np.array(query_embedding)
        
        # 코사인 유사도를 기반으로 유사도 계산
        similarities = []
        for i, vector in enumerate(self.vectors):
            # 필터 함수가 정의되어 있고, 조건에 맞지 않으면 건너뜀
            if filter_func and not filter_func(self.metadata[i]):
                continue
            
            # 코사인 유사도 계산
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))  # 인덱스와 유사도 저장
        
        # 유사도 기준으로 내림차순 정렬
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 상위 k개의 결과 반환
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],                  # 해당 텍스트
                "metadata": self.metadata[idx],           # 해당 메타데이터
                "similarity": float(score)                # 유사도 점수
            })
        
        return results  # 결과 리스트 반환

## Creating Embeddings

In [24]:
def create_embeddings(texts, model="text-embedding-3-small"):
    """
    주어진 텍스트 목록에 대해 임베딩 벡터를 생성합니다.

    Args:
        texts (List[str]): 임베딩을 생성할 입력 텍스트 목록
        model (str): 사용할 임베딩 모델 이름

    Returns:
        List[List[float]]: 생성된 임베딩 벡터 리스트
    """
    # 빈 입력 처리: 텍스트가 없으면 빈 리스트 반환
    if not texts:
        return []
        
    # API 호출 제한을 고려하여 배치 단위로 처리
    batch_size = 100
    all_embeddings = []  # 전체 임베딩 결과 저장 리스트
    
    # 입력 텍스트를 배치 단위로 나누어 처리
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]  # 현재 배치 추출
        
        # 현재 배치에 대해 임베딩 생성 요청
        response = client.embeddings.create(
            model=model,
            input=batch
        )
        
        # 응답에서 임베딩 벡터만 추출
        batch_embeddings = [item.embedding for item in response.data]
        all_embeddings.extend(batch_embeddings)  # 전체 결과에 추가
    
    return all_embeddings  # 전체 임베딩 벡터 반환

## Summarization Function

In [25]:
def generate_page_summary(page_text):
    """
    페이지 단위 텍스트에 대해 간결하고 핵심적인 요약을 생성합니다.

    Args:
        page_text (str): 요약 대상이 되는 페이지의 텍스트 내용

    Returns:
        str: 생성된 요약문
    """
    # 요약 모델에게 역할과 작성 기준을 제시하는 시스템 프롬프트
    system_prompt = """당신은 전문 요약 시스템입니다.
    제공된 텍스트를 기반으로 상세 요약을 작성하세요.
    핵심 주제, 주요 정보, 중요한 사실을 중심으로 요약하되,
    원문보다 간결하면서도 내용을 충분히 파악할 수 있어야 합니다."""

    # 입력 텍스트가 너무 길 경우 잘라서 전달 (토큰 한도 고려)
    max_tokens = 6000
    truncated_text = page_text[:max_tokens] if len(page_text) > max_tokens else page_text

    # OpenAI API를 호출하여 요약 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # 사용할 모델 지정
        messages=[
            {"role": "system", "content": system_prompt},  # 모델에 역할 지시
            {"role": "user", "content": f"다음 텍스트를 요약해 주세요:\n\n{truncated_text}"}  # 사용자가 요청한 텍스트
        ],
        temperature=0.3  # 응답의 일관성과 창의성 균형 조절
    )
    
    # 생성된 요약 결과 반환
    return response.choices[0].message.content

## Hierarchical Document Processing

In [26]:
def process_document_hierarchically(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    문서를 계층적으로 처리하여 요약 인덱스와 상세 인덱스를 생성합니다.

    Args:
        pdf_path (str): PDF 문서 경로
        chunk_size (int): 상세 청크의 크기
        chunk_overlap (int): 청크 간 중첩 길이

    Returns:
        Tuple[SimpleVectorStore, SimpleVectorStore]: 요약용 벡터 저장소, 상세 내용용 벡터 저장소
    """
    # PDF에서 페이지 단위 텍스트 추출
    pages = extract_text_from_pdf(pdf_path)
    
    # 각 페이지에 대해 요약 생성
    print("페이지 요약 생성 중...")
    summaries = []
    for i, page in enumerate(pages):
        print(f"페이지 요약 중: {i+1}/{len(pages)}")
        summary_text = generate_page_summary(page["text"])
        
        # 요약 메타데이터 구성
        summary_metadata = page["metadata"].copy()
        summary_metadata.update({"is_summary": True})  # 요약 플래그 추가
        
        # 요약 텍스트와 메타데이터 저장
        summaries.append({
            "text": summary_text,
            "metadata": summary_metadata
        })
    
    # 각 페이지의 텍스트를 상세 청크로 분할
    detailed_chunks = []
    for page in pages:
        # 해당 페이지의 텍스트를 청크로 나누기
        page_chunks = chunk_text(
            page["text"], 
            page["metadata"], 
            chunk_size, 
            chunk_overlap
        )
        detailed_chunks.extend(page_chunks)  # 전체 청크 리스트에 추가
    
    print(f"총 {len(detailed_chunks)}개의 상세 청크 생성 완료")
    
    # 요약에 대한 임베딩 생성
    print("요약 임베딩 생성 중...")
    summary_texts = [summary["text"] for summary in summaries]
    summary_embeddings = create_embeddings(summary_texts)
    
    # 상세 청크에 대한 임베딩 생성
    print("상세 청크 임베딩 생성 중...")
    chunk_texts = [chunk["text"] for chunk in detailed_chunks]
    chunk_embeddings = create_embeddings(chunk_texts)
    
    # 벡터 저장소 초기화
    summary_store = SimpleVectorStore()
    detailed_store = SimpleVectorStore()
    
    # 요약 벡터 저장소에 항목 추가
    for i, summary in enumerate(summaries):
        summary_store.add_item(
            text=summary["text"],
            embedding=summary_embeddings[i],
            metadata=summary["metadata"]
        )
    
    # 상세 벡터 저장소에 항목 추가
    for i, chunk in enumerate(detailed_chunks):
        detailed_store.add_item(
            text=chunk["text"],
            embedding=chunk_embeddings[i],
            metadata=chunk["metadata"]
        )
    
    print(f"총 {len(summaries)}개의 요약과 {len(detailed_chunks)}개의 청크로 벡터 저장소 생성 완료")
    return summary_store, detailed_store

## Hierarchical Retrieval

In [27]:
def retrieve_hierarchically(query, summary_store, detailed_store, k_summaries=3, k_chunks=5):
    """
    계층적 인덱스를 사용하여 관련 정보를 검색합니다.

    Args:
        query (str): 사용자 질문
        summary_store (SimpleVectorStore): 문서 요약 벡터 저장소
        detailed_store (SimpleVectorStore): 상세 청크 벡터 저장소
        k_summaries (int): 검색할 요약 개수
        k_chunks (int): 요약당 검색할 상세 청크 개수

    Returns:
        List[Dict]: 검색된 청크 목록 (관련도 정보 포함)
    """
    print(f"쿼리에 대한 계층적 검색 수행 중: {query}")
    
    # 쿼리에 대한 임베딩 생성
    query_embedding = create_embeddings(query)
    
    # 1단계: 관련 요약 검색
    summary_results = summary_store.similarity_search(
        query_embedding, 
        k=k_summaries
    )
    
    print(f"관련된 요약 {len(summary_results)}개 검색 완료")
    
    # 관련 요약에서 페이지 번호 수집
    relevant_pages = [result["metadata"]["page"] for result in summary_results]
    
    # 해당 페이지의 청크만 필터링하는 함수 정의
    def page_filter(metadata):
        return metadata["page"] in relevant_pages
    
    # 2단계: 관련 페이지에서만 상세 청크 검색
    detailed_results = detailed_store.similarity_search(
        query_embedding,
        k=k_chunks * len(relevant_pages),
        filter_func=page_filter
    )
    
    print(f"관련 페이지에서 {len(detailed_results)}개의 상세 청크 검색 완료")
    
    # 각 검색 결과에 대응되는 요약 텍스트 추가
    for result in detailed_results:
        page = result["metadata"]["page"]
        matching_summaries = [s for s in summary_results if s["metadata"]["page"] == page]
        if matching_summaries:
            result["summary"] = matching_summaries[0]["text"]
    
    return detailed_results

## Response Generation with Context

In [28]:
def generate_response(query, retrieved_chunks):
    """
    사용자 질문과 검색된 청크를 기반으로 응답을 생성합니다.

    Args:
        query (str): 사용자 질문
        retrieved_chunks (List[Dict]): 계층적 검색으로 찾아낸 관련 청크 목록

    Returns:
        str: 생성된 응답
    """
    # 청크에서 텍스트를 추출하고 문맥(context) 구성
    context_parts = []
    
    for i, chunk in enumerate(retrieved_chunks):
        page_num = chunk["metadata"]["page"]  # 메타데이터에서 페이지 번호 추출
        context_parts.append(f"[Page {page_num}]: {chunk['text']}")  # 페이지 번호 포함한 텍스트 형식화
    
    # 모든 문맥을 하나의 문자열로 합침
    context = "\n\n".join(context_parts)
    
    # AI 어시스턴트에게 역할과 응답 방식에 대한 지침 제공
    system_message = """당신은 제공된 문맥을 바탕으로 질문에 답변하는 유용한 AI 어시스턴트입니다.
    문맥에 포함된 정보를 활용해 정확하게 질문에 답변하세요.
    문맥에 관련 정보가 없는 경우, 그 사실을 명확히 언급하세요.
    특정 정보를 인용할 때는 페이지 번호도 함께 표시하세요."""

    # OpenAI API를 호출하여 응답 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # 사용할 모델 지정
        messages=[
            {"role": "system", "content": system_message},  # 시스템 지시
            {"role": "user", "content": f"문맥:\n\n{context}\n\n질문: {query}"}  # 사용자 메시지 (문맥 + 질문 포함)
        ],
        temperature=0.2  # 응답 일관성을 높이기 위한 낮은 온도 설정
    )
    
    # 생성된 응답 내용 반환
    return response.choices[0].message.content

## Complete RAG Pipeline with Hierarchical Retrieval

In [29]:
def hierarchical_rag(query, pdf_path, chunk_size=1000, chunk_overlap=200, 
                    k_summaries=3, k_chunks=5, regenerate=False):
    """
    계층적 RAG (Retrieval-Augmented Generation) 전체 파이프라인을 실행합니다.

    Args:
        query (str): 사용자 질문
        pdf_path (str): PDF 문서 경로
        chunk_size (int): 상세 청크의 문자 길이
        chunk_overlap (int): 청크 간 중첩 문자 수
        k_summaries (int): 검색할 요약 개수
        k_chunks (int): 요약당 검색할 상세 청크 개수
        regenerate (bool): 벡터 저장소를 새로 생성할지 여부 (True 시 재처리)

    Returns:
        Dict: 쿼리, 응답, 검색된 청크, 요약/청크 개수를 포함한 결과
    """
    # 캐시 파일명 생성 (문서명 기반)
    summary_store_file = f"{os.path.basename(pdf_path)}_summary_store.pkl"
    detailed_store_file = f"{os.path.basename(pdf_path)}_detailed_store.pkl"
    
    # 저장된 벡터 스토어가 없거나 재생성이 필요한 경우 새로 처리
    if regenerate or not os.path.exists(summary_store_file) or not os.path.exists(detailed_store_file):
        print("문서를 처리하고 벡터 저장소를 생성합니다...")
        
        # 문서를 계층적으로 처리하고 벡터 저장소 생성
        summary_store, detailed_store = process_document_hierarchically(
            pdf_path, chunk_size, chunk_overlap
        )
        
        # 생성된 요약 벡터 저장소 저장
        with open(summary_store_file, 'wb') as f:
            pickle.dump(summary_store, f)
        
        # 생성된 상세 벡터 저장소 저장
        with open(detailed_store_file, 'wb') as f:
            pickle.dump(detailed_store, f)
    else:
        # 기존 요약 벡터 저장소 불러오기
        print("기존 벡터 저장소를 불러오는 중...")
        with open(summary_store_file, 'rb') as f:
            summary_store = pickle.load(f)
        
        # 기존 상세 벡터 저장소 불러오기
        with open(detailed_store_file, 'rb') as f:
            detailed_store = pickle.load(f)
    
    # 쿼리를 기반으로 계층적 검색 수행
    retrieved_chunks = retrieve_hierarchically(
        query, summary_store, detailed_store, k_summaries, k_chunks
    )
    
    # 검색된 청크를 기반으로 응답 생성
    response = generate_response(query, retrieved_chunks)
    
    # 최종 결과 반환 (쿼리, 응답, 검색 결과, 벡터 저장소 크기 포함)
    return {
        "query": query,
        "response": response,
        "retrieved_chunks": retrieved_chunks,
        "summary_count": len(summary_store.texts),
        "detailed_count": len(detailed_store.texts)
    }

## Standard (Non-Hierarchical) RAG for Comparison

In [30]:
def standard_rag(query, pdf_path, chunk_size=1000, chunk_overlap=200, k=15):
    """
    계층적 검색 없이 단순한 RAG 파이프라인을 실행합니다.

    Args:
        query (str): 사용자 질문
        pdf_path (str): PDF 문서 경로
        chunk_size (int): 각 청크의 크기 (문자 단위)
        chunk_overlap (int): 청크 간 중첩 길이
        k (int): 검색할 상위 청크 개수

    Returns:
        Dict: 쿼리, 응답, 검색된 청크를 포함한 결과
    """
    # PDF 문서에서 텍스트를 페이지 단위로 추출
    pages = extract_text_from_pdf(pdf_path)
    
    # 전체 페이지에서 바로 청크 생성
    chunks = []
    for page in pages:
        # 해당 페이지의 텍스트를 청크로 분할
        page_chunks = chunk_text(
            page["text"], 
            page["metadata"], 
            chunk_size, 
            chunk_overlap
        )
        chunks.extend(page_chunks)  # 생성된 청크 추가
    
    print(f"기본 RAG용 청크 {len(chunks)}개 생성 완료")
    
    # 벡터 저장소 초기화
    store = SimpleVectorStore()
    
    # 각 청크에 대해 임베딩 생성
    print("청크 임베딩 생성 중...")
    texts = [chunk["text"] for chunk in chunks]
    embeddings = create_embeddings(texts)
    
    # 임베딩과 함께 청크를 저장소에 추가
    for i, chunk in enumerate(chunks):
        store.add_item(
            text=chunk["text"],
            embedding=embeddings[i],
            metadata=chunk["metadata"]
        )
    
    # 쿼리에 대한 임베딩 생성
    query_embedding = create_embeddings(query)
    
    # 쿼리 임베딩을 기반으로 가장 유사한 청크 검색
    retrieved_chunks = store.similarity_search(query_embedding, k=k)
    print(f"기본 RAG로 {len(retrieved_chunks)}개의 청크 검색 완료")
    
    # 검색된 청크를 바탕으로 응답 생성
    response = generate_response(query, retrieved_chunks)
    
    # 결과 반환
    return {
        "query": query,
        "response": response,
        "retrieved_chunks": retrieved_chunks
    }

## Evaluation Functions

In [31]:
def compare_approaches(query, pdf_path, reference_answer=None):
    """
    계층형 RAG과 일반 RAG 접근 방식을 비교합니다.

    Args:
        query (str): 사용자 질문
        pdf_path (str): PDF 문서 경로
        reference_answer (str, optional): 평가 기준이 될 정답 (선택 사항)

    Returns:
        Dict: 두 방식의 응답 및 비교 분석 결과
    """
    print(f"\n=== RAG 방식 비교 시작: 질문 → {query} ===")
    
    # 계층형 RAG 실행
    print("\n[1] 계층형 RAG 실행 중...")
    hierarchical_result = hierarchical_rag(query, pdf_path)
    hier_response = hierarchical_result["response"]
    
    # 일반 RAG 실행
    print("\n[2] 일반 RAG 실행 중...")
    standard_result = standard_rag(query, pdf_path)
    std_response = standard_result["response"]
    
    # 두 응답을 비교 (참조 정답이 있는 경우 함께 평가)
    comparison = compare_responses(query, hier_response, std_response, reference_answer)
    
    # 결과 딕셔너리로 정리하여 반환
    return {
        "query": query,  # 사용자 질문
        "hierarchical_response": hier_response,  # 계층형 RAG의 응답
        "standard_response": std_response,  # 일반 RAG의 응답
        "reference_answer": reference_answer,  # 기준 정답 (선택 사항)
        "comparison": comparison,  # 두 응답 간 비교 분석 결과
        "hierarchical_chunks_count": len(hierarchical_result["retrieved_chunks"]),  # 계층형 RAG에서 검색된 청크 수
        "standard_chunks_count": len(standard_result["retrieved_chunks"])  # 일반 RAG에서 검색된 청크 수
    }

In [32]:
def compare_responses(query, hierarchical_response, standard_response, reference=None):
    """
    계층형 RAG과 일반 RAG의 응답을 비교 분석합니다.

    Args:
        query (str): 사용자 질문
        hierarchical_response (str): 계층형 RAG의 응답
        standard_response (str): 일반 RAG의 응답
        reference (str, optional): 기준 정답 (선택 사항)

    Returns:
        str: 두 응답에 대한 비교 분석 결과
    """
    # 평가 모델에게 역할과 기준을 안내하는 시스템 프롬프트
    system_prompt = """당신은 정보 검색 시스템 평가 전문가입니다. 
    동일한 질문에 대해 계층형 검색과 일반 검색을 통해 생성된 두 개의 응답을 비교하세요.

    다음 기준을 바탕으로 평가하십시오:
    1. 정확성: 어떤 응답이 더 사실에 기반한 정확한 정보를 제공하는가?
    2. 포괄성: 어떤 응답이 질문의 여러 측면을 더 잘 다루는가?
    3. 논리성: 어떤 응답이 더 논리적이고 자연스러운 흐름을 가지는가?
    4. 페이지 참조: 어떤 응답이 페이지 정보를 더 잘 활용하고 있는가?

    각 접근 방식의 장단점을 구체적으로 분석해 주세요."""

    # 사용자 프롬프트 구성: 질문과 두 응답 포함
    user_prompt = f"""질문:
    {query}

    [계층형 RAG 응답]
    {hierarchical_response}

    [일반 RAG 응답]
    {standard_response}"""

    # 기준 정답이 있을 경우 포함
    if reference:
        user_prompt += f"""

    [기준 정답]
    {reference}"""

    # 최종 지시문 추가
    user_prompt += """

    이 두 응답에 대해 어떤 방식이 더 나은 성능을 보였는지 구체적으로 비교 분석해 주세요."""

    # OpenAI API 호출하여 비교 분석 요청
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0  # 일관되고 분석적인 응답을 위해 낮은 온도 설정
    )
    
    # 생성된 비교 분석 결과 반환
    return response.choices[0].message.content

In [33]:
def run_evaluation(pdf_path, test_queries, reference_answers=None):
    """
    여러 테스트 쿼리에 대해 전체 RAG 평가를 수행합니다.

    Args:
        pdf_path (str): 평가 대상 PDF 문서 경로
        test_queries (List[str]): 테스트 질문 목록
        reference_answers (List[str], optional): 각 질문에 대한 기준 정답 (선택 사항)

    Returns:
        Dict: 쿼리별 결과 및 전체 분석을 포함한 평가 결과
    """
    results = []  # 개별 평가 결과를 저장할 리스트 초기화

    # 각 테스트 질문에 대해 반복 수행
    for i, query in enumerate(test_queries):
        print(f"질문: {query}")  # 현재 질문 출력

        # 기준 정답이 존재할 경우 연결
        reference = None
        if reference_answers and i < len(reference_answers):
            reference = reference_answers[i]

        # 계층형 RAG vs 일반 RAG 응답 비교
        result = compare_approaches(query, pdf_path, reference)
        results.append(result)  # 결과 리스트에 추가
    
    # 전체 평가 결과에 대한 종합 분석 생성
    overall_analysis = generate_overall_analysis(results)

    return {
        "results": results,  # 쿼리별 상세 결과
        "overall_analysis": overall_analysis  # 전체 평가 요약 분석
    }

In [34]:
def generate_overall_analysis(results):
    """
    다수의 쿼리 평가 결과를 기반으로 전체적인 성능 분석을 생성합니다.

    Args:
        results (List[Dict]): 개별 쿼리 평가 결과 목록

    Returns:
        str: 계층형 RAG과 일반 RAG 간의 전체 비교 분석 결과
    """
    # 평가 모델에게 전체 분석 기준을 안내하는 시스템 프롬프트
    system_prompt = """당신은 정보 검색 시스템 평가 전문가입니다.
    여러 테스트 쿼리 결과를 기반으로, 계층형 RAG과 일반 RAG의 전반적인 성능을 비교 분석하세요.

    다음 항목에 초점을 맞춰 작성하십시오:
    1. 계층형 RAG이 더 우수한 상황과 그 이유
    2. 일반 RAG이 더 우수한 상황과 그 이유
    3. 각 접근 방식의 전반적인 강점과 한계
    4. 사용 상황에 따른 접근 방식 추천"""

    # 각 쿼리별 요약을 텍스트로 구성
    evaluations_summary = ""
    for i, result in enumerate(results):
        evaluations_summary += f"질문 {i+1}: {result['query']}\n"
        evaluations_summary += f"계층형 청크 수: {result['hierarchical_chunks_count']}, 일반 청크 수: {result['standard_chunks_count']}\n"
        evaluations_summary += f"비교 요약: {result['comparison'][:200]}...\n\n"

    # 사용자 프롬프트: 모든 쿼리 평가 결과를 요약한 내용 포함
    user_prompt = f"""아래는 총 {len(results)}개의 쿼리에 대해 계층형 RAG과 일반 RAG을 비교한 평가 요약입니다.
    이 내용을 바탕으로 두 접근 방식에 대한 전체 분석을 작성해 주세요:

    {evaluations_summary}

    계층형 RAG과 일반 RAG의 상대적 강점과 약점을 종합적으로 분석하고,
    각 접근 방식을 어떤 상황에서 사용하는 것이 적절한지에 대한 제언도 포함해 주세요."""

    # OpenAI API 호출을 통해 종합 분석 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},  # 시스템 역할 지시
            {"role": "user", "content": user_prompt}  # 사용자 프롬프트 전달
        ],
        temperature=0  # 일관된 분석을 위한 낮은 온도
    )
    
    # 생성된 분석 내용 반환
    return response.choices[0].message.content

## Evaluation of Hierarchical and Standard RAG Approaches

In [35]:
# AI 정보가 담긴 PDF 문서 경로
pdf_path = "dataset/AI_Understanding.pdf"

# 계층형 RAG 성능 확인용 예시 질문
query = "자연어 처리에서 트랜스포머 모델의 주요 활용 사례는 무엇인가요?"
result = hierarchical_rag(query, pdf_path)

print("\n=== 응답 결과 ===")
print(result["response"])

# 공식 비교 평가용 테스트 쿼리 (1개만 테스트)
test_queries = [
    "트랜스포머는 순차 데이터를 RNN과 어떻게 다르게 처리하나요?"
]

# 기준 정답 (reference answer)
reference_answers = [
    "트랜스포머는 순차 데이터를 순차적으로 처리하는 RNN과 달리, self-attention 메커니즘을 통해 모든 토큰을 병렬로 처리합니다. 이 구조는 긴 거리의 의존성을 더 효과적으로 포착할 수 있으며, 학습 시 병렬화가 가능해 효율적입니다. 또한 트랜스포머는 긴 시퀀스를 처리할 때 RNN보다 기울기 소실 문제에서 자유롭습니다."
]

# 계층형 RAG vs 일반 RAG 비교 평가 실행
evaluation_results = run_evaluation(
    pdf_path=pdf_path,
    test_queries=test_queries,
    reference_answers=reference_answers
)

# 전체 평가 요약 분석 출력
print("\n***전체 평가 요약***")
print(evaluation_results["overall_analysis"])

문서를 처리하고 벡터 저장소를 생성합니다...
dataset/AI_Understanding.pdf에서 텍스트를 추출하는 중...
내용이 있는 페이지 15개를 추출했습니다.
페이지 요약 생성 중...
페이지 요약 중: 1/15
페이지 요약 중: 2/15
페이지 요약 중: 3/15
페이지 요약 중: 4/15
페이지 요약 중: 5/15
페이지 요약 중: 6/15
페이지 요약 중: 7/15
페이지 요약 중: 8/15
페이지 요약 중: 9/15
페이지 요약 중: 10/15
페이지 요약 중: 11/15
페이지 요약 중: 12/15
페이지 요약 중: 13/15
페이지 요약 중: 14/15
페이지 요약 중: 15/15
총 30개의 상세 청크 생성 완료
요약 임베딩 생성 중...
상세 청크 임베딩 생성 중...
총 15개의 요약과 30개의 청크로 벡터 저장소 생성 완료
쿼리에 대한 계층적 검색 수행 중: 자연어 처리에서 트랜스포머 모델의 주요 활용 사례는 무엇인가요?


/tmp/ipykernel_3601049/3167212948.py:62: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  "similarity": float(score)                # 유사도 점수


관련된 요약 3개 검색 완료
관련 페이지에서 6개의 상세 청크 검색 완료

=== 응답 결과 ===
제공된 문맥에는 자연어 처리(NLP)에서 트랜스포머 모델의 주요 활용 사례에 대한 정보가 포함되어 있지 않습니다. 따라서 이 질문에 대한 구체적인 답변을 제공할 수 없습니다. 트랜스포머 모델은 일반적으로 기계 번역, 텍스트 요약, 감정 분석 등 다양한 NLP 작업에 활용되지만, 문맥에서는 이러한 내용이 언급되지 않았습니다.
질문: 트랜스포머는 순차 데이터를 RNN과 어떻게 다르게 처리하나요?

=== RAG 방식 비교 시작: 질문 → 트랜스포머는 순차 데이터를 RNN과 어떻게 다르게 처리하나요? ===

[1] 계층형 RAG 실행 중...
기존 벡터 저장소를 불러오는 중...
쿼리에 대한 계층적 검색 수행 중: 트랜스포머는 순차 데이터를 RNN과 어떻게 다르게 처리하나요?
관련된 요약 3개 검색 완료
관련 페이지에서 6개의 상세 청크 검색 완료

[2] 일반 RAG 실행 중...
dataset/AI_Understanding.pdf에서 텍스트를 추출하는 중...
내용이 있는 페이지 15개를 추출했습니다.
기본 RAG용 청크 30개 생성 완료
청크 임베딩 생성 중...
기본 RAG로 15개의 청크 검색 완료

***전체 평가 요약***
### 계층형 RAG과 일반 RAG의 비교 분석

#### 1. 계층형 RAG이 더 우수한 상황과 그 이유
계층형 RAG는 정보의 구조적 계층을 활용하여 더 깊이 있는 이해를 제공할 수 있는 경우에 우수합니다. 예를 들어, 복잡한 주제나 다단계 질문에 대해 계층형 RAG는 관련 정보를 더 잘 조직하고, 각 계층에서 중요한 세부사항을 강조할 수 있습니다. 이는 특히 기술적이거나 학문적인 질문에서 유용하며, 사용자가 원하는 정보의 맥락을 명확히 전달할 수 있습니다. 

또한, 계층형 RAG는 정보의 중요도를 평가하여 더 관련성 높은 응답을 생성할 수 있는 장점이 있습니다. 예를 들어, 질문이 "트랜스포머는 순차 데이터를 RNN과 어떻게 